In [0]:
import jieba
from urllib.request import urlretrieve
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.txt.big")
jieba.set_dictionary("dict.txt.big")

In [0]:
def commentcut(s):
    s = s.replace(" ", "").replace(":", "").replace("n", "").replace("r", "").replace("r", "...")
    s = " ".join(jieba.cut(s))
    
    return s

In [0]:
import pandas as pd

df = pd.read_table("drive/My Drive/pos_neg/pos_chi.txt")
len(df["content"][23576])
for i , j in enumerate(df["content"]):
    if len(j)<9:
        df = df.drop([i])
df.reset_index(drop=True, inplace=True)


In [11]:
df

,content
0,好 感謝一樓大大，我來裝看看！感恩
1,thx u12 life
2,S6 Lite 入門就 64G，還送不錯的筆跟套
3,S5E 螢幕好很多 常使用就買比較不傷眼的
4,你有買很好啊！不要像有個人自稱有買note1
...,...
19889,看成apple控告高通性侵@@ 11/3
19890,原po好帥人超好快幫我改啦嗚嗚嗚嗚 11/26 19
19891,實蠻快的，服務也很棒 11/
19892,i8真的只是煙霧彈@@ 1


In [12]:
import re
punctuation = """@/1 ，！？｡＂"＃'~,、+!.\＄％＆*＇（）＊＋－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘'‛“”„‟…‧﹏"""
re_punctuation = "[{}]+".format(punctuation)
for i in range(0, len(df["content"])):
    df["content"][i] = re.sub(re_punctuation, "", df["content"][i])
df


,content
0,好感謝一樓大大我來裝看看感恩
1,thxu2life
2,S6Lite入門就64G還送不錯的筆跟套
3,S5E螢幕好很多常使用就買比較不傷眼的
4,你有買很好啊不要像有個人自稱有買note
...,...
19889,看成apple控告高通性侵3
19890,原po好帥人超好快幫我改啦嗚嗚嗚嗚269
19891,實蠻快的服務也很棒
19892,i8真的只是煙霧彈


In [13]:
df["content"] = df["content"].apply(commentcut)
df

Building prefix dict from /content/dict.txt.big ...
Dumping model to file cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
Loading model cost 1.609 seconds.
Prefix dict has been built successfully.


,content
0,好 感謝 一樓 大大 我來裝 看看 感恩
1,thxu2life
2,S6Lite 入門 就 64G 還送 不錯 的 筆跟套
3,S5E 螢幕 好 很多 常 使用 就 買 比較 不 傷眼 的
4,你 有 買 很 好 啊 不要 像 有 個人 自稱 有 買 ote
...,...
19889,看成 apple 控告 高通 性侵 3
19890,原 po 好帥 人 超好 快 幫 我 改 啦 嗚嗚 嗚嗚 269
19891,實蠻 快 的 服務 也 很棒
19892,i8 真的 只是 煙霧彈


In [0]:
# Import the necessary modules
import pickle
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences


# 匯入字典
with open('drive/My Drive/tokenizer.pickle', 'rb') as f:
    tok = pickle.load(f)
with open('drive/My Drive/label_dict.pk', 'rb') as f:
    output_dictionary = pickle.load(f)

# 載入模型
model_save_path = 'drive/My Drive/lstm_model.h5'
lstm_model = load_model(model_save_path)

In [0]:
for i in df["content"][0:100]:
  print(i)

In [15]:
pos = 0
neg = 0
count = 0
for i in df["content"][0:100]:
    # 資料預處理
    # sent = ["S5E 螢幕 好 很多 常 使用 就 買 比較 不 傷眼 的"]
    # tok = Tokenizer(num_words=20000)
    # tok.fit_on_texts(x)

    x = tok.texts_to_sequences([i])
    x_pad = pad_sequences(x, maxlen=128)
    # print(x_pad)

    # 模型預測
    y_predict = lstm_model.predict(x_pad)
    label_dict = {v:k for k,v in output_dictionary.items()}
    # print('輸入語句: %s' % sent)
    # print('情感預測結果: %s' % label_dict[np.argmax(y_predict)])
    if np.argmax(y_predict) == 0:
        neg += 1
        print(i)
    elif np.argmax(y_predict) == 1:
        pos += 1
    count += 1
print(f"在{count}則正向評論中，您的預測準度為:", 100*round(float(pos/count), 2),"%")

好 感謝 一樓 大大 我來裝 看看 感恩
thxu2life
S6Lite 入門 就 64G 還送 不錯 的 筆跟套
SE2 就 續航 差小 螢幕 大 尺寸 高效 時會 過熱 降頻 大多
樓上 說 的 太好了 那 我們 應該 先來 分中階 低階
當 了 四次 多 打 看錯
哈哈哈 釣出 一堆 超 好笑
魚兒 效果 卓越 呢
ipad 也 是 大 螢幕 iphoe 啊 不能 取代 筆電 而且 返回
台版 雖然 是 高通 但是 比較 喜歡 韓版 的 特殊 色 XD
感謝 各位 的 建議 我會 去 買 防水 套 來用 的
買 防水 套 才 是 真 防水 任何 標榜 防水 的 手機 受潮 維修
下次 可 以來 個 凹面 螢幕 進去 嗎
曲面 視覺 很爽 又 漂亮 用 習慣 就 不 容易 誤觸 了
剛剛 狂玩好 有趣
Note9 已經 算是 續航 很棒 的 三星 旗艦 了
我 Adoid9 成功 開卡 拿到 全家 禮券
mobilepay 討論 很 多次 了 窩 就是 有鎖 地區
感覺 很 不錯 耶早 用 早 感受 5G
微風 南山 很快 啦 3
S 系列 打 年輕 和 小資 族群 這 顏色 很 適合 啊 用 重色 就
恭喜 入手 我 也 拿 了 一台 晴空 藍 但 我 覺得 最 好看 的 是
微軟 喔 加油 捏 xboxlik
不用 切 回來 所以 是 開 了 payapp 他 就 會 自己 優先 用
而是 得 手動 操作 加值 每次 你 可以 選 不同 卡
iOS 流口水
可以 同 NFCSIM 機器 加值 或是 超商 零錢 加值 都行
太棒了 台北 是 三星 的 城市
保護 貼 應該 好找 很多
早 在 用 遠傳 的 fc 悠遊 卡 了 很 方便 而且 可以 設定 自
Ulta 相機 沒 調教 好 真的 蠻 意外 的 加上 價格 搞不好
含 贈品 含 兩年 保含 螢幕 保險 當然 會 比 國外 高
考驗 信仰 的 時候 來 了
某 sh 真好玩 什麼 粉 就 等於 通粉 哈哈
高 售價 也許 可以 拉長 換機 期 比較 環保 讚 讚
在100則正向評論中，您的預測準度為: 65.0 %


In [0]:
y_predict


array([[0.7310681 , 0.26893187]], dtype=float32)

In [0]:
s="123456123456"
s.rsplit("5", 1)

['1234561234', '6']